In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from imageio import imread
import numpy as np
import os
import math
import PIL
from time import time

In [2]:
use_random=True
# Tensor and PIL utils

def pil_loader(path):
    with open(path, 'rb') as f:
        img = PIL.Image.open(f)
        return img.convert('RGB')

def tensor_resample(tensor, dst_size, mode='bilinear'):
    return F.interpolate(tensor, dst_size, mode=mode, align_corners=False)

def pil_resize_short_edge_to(pil, trg_size):
    short_w = pil.width < pil.height
    ar_resized_short = (trg_size / pil.width) if short_w else (trg_size / pil.height)
    resized = pil.resize((int(pil.width * ar_resized_short), int(pil.height * ar_resized_short)), PIL.Image.BICUBIC)
    return resized

def pil_resize_long_edge_to(pil, trg_size):
    short_w = pil.width < pil.height
    ar_resized_long = (trg_size / pil.height) if short_w else (trg_size / pil.width)
    resized = pil.resize((int(pil.width * ar_resized_long), int(pil.height * ar_resized_long)), PIL.Image.BICUBIC)
    return resized

def np_to_pil(npy):
    return PIL.Image.fromarray(npy.astype(np.uint8))

def pil_to_np(pil):
    return np.array(pil)

In [3]:
def tensor_to_np(tensor, cut_dim_to_3=True):
    if len(tensor.shape) == 4:
        if cut_dim_to_3:
            tensor = tensor[0]
        else:
            return tensor.data.cpu().numpy().transpose((0, 2, 3, 1))
    return tensor.data.cpu().numpy().transpose((1,2,0))

def np_to_tensor(npy, space):
    if space == 'vgg':
        return np_to_tensor_correct(npy)
    return (torch.Tensor(npy.astype(np.float64) / 127.5) - 1.0).permute((2,0,1)).unsqueeze(0)

def np_to_tensor_correct(npy):
    pil = np_to_pil(npy)
    transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform(pil).unsqueeze(0)

In [4]:
# Laplacian Pyramid

def laplacian(x):
    # x - upsample(downsample(x))
    return x - tensor_resample(tensor_resample(x, [x.shape[2] // 2, x.shape[3] // 2]), [x.shape[2], x.shape[3]])

def make_laplace_pyramid(x, levels):
    pyramid = []
    current = x
    for i in range(levels):
        pyramid.append(laplacian(current))
        current = tensor_resample(current, (max(current.shape[2] // 2,1), max(current.shape[3] // 2,1)))
    pyramid.append(current)
    return pyramid

def fold_laplace_pyramid(pyramid):
    current = pyramid[-1]
    for i in range(len(pyramid)-2, -1, -1): # iterate from len-2 to 0
        up_h, up_w = pyramid[i].shape[2], pyramid[i].shape[3]
        current = pyramid[i] + tensor_resample(current, (up_h,up_w))
    return current

In [5]:
def sample_indices(feat_content, feat_style_all, r, ri, xx, xy, yx):

    indices = None
    const = 128**2 # 32k or so

    feat_style =  feat_style_all[ri]

    for i in range(len(feat_style)):
        
        feat_cont = feat_content[i]
        d = feat_style[i].size(1)
        feat_style_st = feat_style[i].view(1,d,-1,1)
        big_size = feat_cont.shape[2] * feat_cont.shape[3] # num feaxels

        stride_x = int(max(math.floor(math.sqrt(big_size//const)),1))
        offset_x = np.random.randint(stride_x)
        stride_y = int(max(math.ceil(math.sqrt(big_size//const)),1))
        offset_y = np.random.randint(stride_y)
        xx_arr, xy_arr = np.meshgrid(np.arange(feat_cont.shape[2])[offset_x::stride_x], np.arange(feat_cont.shape[3])[offset_y::stride_y])

        xx_arr = np.expand_dims(xx_arr.flatten(),1)
        xy_arr = np.expand_dims(xy_arr.flatten(),1)
        xc = np.concatenate([xx_arr,xy_arr], 1)

        region_mask = r

        try:
            xc = xc[region_mask[xy_arr[:,0],xx_arr[:,0]], :]
        except:
            region_mask = region_mask[:,:]
            xc = xc[region_mask[xy_arr[:,0],xx_arr[:,0]], :]
        
        xx[ri].append(xc[:,0])
        xy[ri].append(xc[:,1])

        feat_result = np.arange(feat_style_st.size(2)).astype(np.int32)
        yx[ri].append(feat_result)

def get_feature_indices(xx_dict, xy_dict, yx_dict, ri=0, i=0, cnt=32**2):

    xx = xx_dict[ri][i][:cnt]
    xy = xy_dict[ri][i][:cnt]
    yx = yx_dict[ri][i][:cnt]

    return xx, xy, yx


In [6]:
def spatial_feature_extract(feat_result, feat_content, xx, xy):

    l2, l3 = [], []
    device = feat_result[0].device

    # for each extracted layer
    for i in range(len(feat_result)):
        fr = feat_result[i]
        fc = feat_content[i]

        # hack to detect reduced scale
        if i>0 and feat_result[i-1].size(2) > feat_result[i].size(2):
            xx = xx/2.0
            xy = xy/2.0


        # go back to ints and get residual
        xxm = np.floor(xx).astype(np.float32)
        xxr = xx - xxm

        xym = np.floor(xy).astype(np.float32)
        xyr = xy - xym

        # do bilinear resample
        w00 = torch.from_numpy((1.-xxr)*(1.-xyr)).float().view(1, 1, -1, 1).to(device)
        w01 = torch.from_numpy((1.-xxr)*xyr).float().view(1, 1, -1, 1).to(device)
        w10 = torch.from_numpy(xxr*(1.-xyr)).float().view(1, 1, -1, 1).to(device)
        w11 = torch.from_numpy(xxr*xyr).float().view(1, 1, -1, 1).to(device)

        xxm = np.clip(xxm.astype(np.int32),0,fr.size(2)-1)
        xym = np.clip(xym.astype(np.int32),0,fr.size(3)-1)

        s00 = xxm*fr.size(3)+xym
        s01 = xxm*fr.size(3)+np.clip(xym+1,0,fr.size(3)-1)
        s10 = np.clip(xxm+1,0,fr.size(2)-1)*fr.size(3)+(xym)
        s11 = np.clip(xxm+1,0,fr.size(2)-1)*fr.size(3)+np.clip(xym+1,0,fr.size(3)-1)

        fr = fr.view(1,fr.size(1),fr.size(2)*fr.size(3),1)
        fr = fr[:,:,s00,:].mul_(w00).add_(fr[:,:,s01,:].mul_(w01)).add_(fr[:,:,s10,:].mul_(w10)).add_(fr[:,:,s11,:].mul_(w11))

        fc = fc.view(1,fc.size(1),fc.size(2)*fc.size(3),1)
        fc = fc[:,:,s00,:].mul_(w00).add_(fc[:,:,s01,:].mul_(w01)).add_(fc[:,:,s10,:].mul_(w10)).add_(fc[:,:,s11,:].mul_(w11))

        l2.append(fr)
        l3.append(fc)

    x_st = torch.cat([li.contiguous() for li in l2],1)
    c_st = torch.cat([li.contiguous() for li in l3],1)

    xx = torch.from_numpy(xx).view(1,1,x_st.size(2),1).float().to(device)
    yy = torch.from_numpy(xy).view(1,1,x_st.size(2),1).float().to(device)
    
    x_st = torch.cat([x_st,xx,yy],1)
    c_st = torch.cat([c_st,xx,yy],1)
    return x_st, c_st

In [7]:
def rgb_to_yuv(rgb):
    C = torch.Tensor([[0.577350,0.577350,0.577350],[-0.577350,0.788675,-0.211325],[-0.577350,-0.211325,0.788675]]).to(rgb.device)
    yuv = torch.mm(C,rgb)
    return yuv

def pairwise_distances_cos(x, y):
    x_norm = torch.sqrt((x**2).sum(1).view(-1, 1))
    y_t = torch.transpose(y, 0, 1)
    y_norm = torch.sqrt((y**2).sum(1).view(1, -1))
    dist = 1.-torch.mm(x, y_t)/x_norm/y_norm
    return dist

def pairwise_distances_sq_l2(x, y):
    x_norm = (x**2).sum(1).view(-1, 1)
    y_t = torch.transpose(y, 0, 1)
    y_norm = (y**2).sum(1).view(1, -1)
    dist = x_norm + y_norm - 2.0 * torch.mm(x, y_t)
    return torch.clamp(dist, 1e-5, 1e5)/x.size(1)

In [8]:
def create_mask_from_image(image, ignore_color=[0, 0, 0]):
    """
    Create a mask from an image, where pixels matching the ignore_color are set to 0, and others to 1.

    :param image_path: Path to the input image.
    :param ignore_color: Color to be ignored, default is black ([0, 0, 0]).
    :return: Mask tensor of shape (1, H, W), where 1 indicates important areas and 0 indicates areas to ignore.
    """

    # Check if the image is grayscale or RGB
    if len(image.shape) == 2:  # Grayscale image
        mask = image != ignore_color[0]
    else:  # RGB image
        mask = np.all(image != ignore_color, axis=-1)

    mask = torch.from_numpy(mask).unsqueeze(0).float()

    return mask

In [9]:
def extract_regions(content_path, style_path):
    s_regions = imread(style_path).transpose(1,0,2)
    c_regions = imread(content_path).transpose(1,0,2)

    color_codes,c1 = np.unique(s_regions.reshape(-1, s_regions.shape[2]), axis=0,return_counts=True)

    color_codes = color_codes[c1>10000]

    c_out = []
    s_out = []

    for c in color_codes:
        c_expand =  np.expand_dims(np.expand_dims(c,0),0)
        
        s_mask = np.equal(np.sum(s_regions - c_expand,axis=2),0).astype(np.float32)
        c_mask = np.equal(np.sum(c_regions - c_expand,axis=2),0).astype(np.float32)

        s_out.append(s_mask)
        c_out.append(c_mask)

    return [c_out,s_out]

In [10]:
def distmat(x, y, cos_d=True):
    if cos_d:
        M = pairwise_distances_cos(x, y)
    else:
        M = torch.sqrt(pairwise_distances_sq_l2(x, y))
    return M

In [11]:
def content_loss(feat_result, feat_content):
    d = feat_result.size(1)

    X = feat_result.transpose(0,1).contiguous().view(d,-1).transpose(0,1)
    Y = feat_content.transpose(0,1).contiguous().view(d,-1).transpose(0,1)

    Y = Y[:,:-2]
    X = X[:,:-2]
    # X = X.t()
    # Y = Y.t()

    Mx = distmat(X, X)
    Mx = Mx#/Mx.sum(0, keepdim=True)

    My = distmat(Y, Y)
    My = My#/My.sum(0, keepdim=True)

    d = torch.abs(Mx-My).mean()# * X.shape[0]
    return d


In [12]:
def style_loss(X, Y, cos_d=True):
    d = X.shape[1]

    if d == 3:
        X = rgb_to_yuv(X.transpose(0,1).contiguous().view(d,-1)).transpose(0,1)
        Y = rgb_to_yuv(Y.transpose(0,1).contiguous().view(d,-1)).transpose(0,1)
    else:
        X = X.transpose(0,1).contiguous().view(d,-1).transpose(0,1)
        Y = Y.transpose(0,1).contiguous().view(d,-1).transpose(0,1)

    # Relaxed EMD
    CX_M = distmat(X, Y, cos_d=True)

    if d==3: CX_M = CX_M + distmat(X, Y, cos_d=False)

    m1, m1_inds = CX_M.min(1)
    m2, m2_inds = CX_M.min(0)

    remd = torch.max(m1.mean(), m2.mean())

    return remd

In [13]:
def moment_loss(X, Y, moments=[1,2]):
    loss = 0.
    d = X.size(1)
    # X = X.squeeze().t()
    # Y = Y.squeeze().t()

    Xo = X.transpose(0,1).contiguous().view(d,-1).transpose(0,1)
    Yo = Y.transpose(0,1).contiguous().view(d,-1).transpose(0,1)

    splits = [Xo.size(1)]

    cb = 0
    ce = 0
    for i in range(len(splits)):
        ce = cb + splits[i]
        X = Xo[:,cb:ce]
        Y = Yo[:,cb:ce]
        cb = ce

        mu_x = torch.mean(X,0,keepdim=True)
        mu_y = torch.mean(Y,0,keepdim=True)
        mu_d = torch.abs(mu_x-mu_y).mean()

        if 1 in moments:
            # print(mu_x.shape)
            loss = loss + mu_d

        if 2 in moments:

            sig_x = torch.mm((X-mu_x).transpose(0,1), (X-mu_x))/X.size(0)
            sig_y = torch.mm((Y-mu_y).transpose(0,1), (Y-mu_y))/Y.size(0)

            sig_d = torch.abs(sig_x-sig_y).mean()

            # print(X_cov.shape)
            # exit(1)
            loss = loss + sig_d

    return loss

In [14]:
def calculate_loss(feat_result, feat_content, feat_style, xx_dict, xy_dict, yx_dict, content_weight, regions, moment_weight=1.0):
    # spatial feature extract
    num_locations = 1024
    loss_total = 0.

    for ri in range(len(regions[0])):
        xx, xy, yx = get_feature_indices(xx_dict, xy_dict, yx_dict, ri=ri, cnt=num_locations)
        spatial_result, spatial_content = spatial_feature_extract(feat_result, feat_content, xx, xy)

        loss_content = content_loss(spatial_result, spatial_content)

        d = feat_style[ri][0].shape[0]
        spatial_style = feat_style[ri][0].view(1, d, -1, 1)

        feat_max = 3+2*64+128*2+256*3+512*2 # (sum of all extracted channels)

        loss_remd = style_loss(spatial_result[:, :feat_max, :, :], spatial_style[:, :feat_max, :, :])

        loss_moment = moment_loss(spatial_result[:,:-2,:,:], spatial_style, moments=[1,2]) # -2 is so that it can fit?
        # palette matching
        content_weight_frac = 1./max(content_weight,1.)
        loss_moment += content_weight_frac * style_loss(spatial_result[:,:3,:,:], spatial_style[:,:3,:,:])
        
        loss_style = loss_remd + moment_weight * loss_moment
        print(f'Style: {loss_style.item():.3f}, Content: {loss_content.item():.3f}')

        style_weight = 1.0 + moment_weight
        loss_total += (content_weight * loss_content + loss_style) / (content_weight + style_weight)

    return loss_total/len(xx_dict.keys())

In [15]:
RESAMPLE_FREQ = 1

class Vgg16_Extractor(nn.Module):
    def __init__(self, space):
        super().__init__()
        self.vgg_layers = models.vgg16(pretrained=True).features

        for param in self.parameters():
            param.requires_grad = False
        self.capture_layers = [1,3,6,8,11,13,15,22,29]
        self.space = space
        
    def forward_base(self, x):
        feat = [x]
        for i in range(len(self.vgg_layers)):
            x = self.vgg_layers[i](x)
            if i in self.capture_layers: feat.append(x)
        return feat

    def forward(self, x):
        if self.space != 'vgg':
            x = (x + 1.) / 2.
            x = x - (torch.Tensor([0.485, 0.456, 0.406]).to(x.device).view(1, -1, 1, 1))
            x = x / (torch.Tensor([0.229, 0.224, 0.225]).to(x.device).view(1, -1, 1, 1))
        feat = self.forward_base(x)
        return feat
    
    def forward_samples_hypercolumn(self, X, samps=100):
        feat = self.forward(X)

        xx,xy = np.meshgrid(np.arange(X.shape[2]), np.arange(X.shape[3]))
        xx = np.expand_dims(xx.flatten(),1)
        xy = np.expand_dims(xy.flatten(),1)
        xc = np.concatenate([xx,xy],1)
        
        samples = min(samps,xc.shape[0])

        np.random.shuffle(xc)
        xx = xc[:samples,0]
        yy = xc[:samples,1]

        feat_samples = []
        for i in range(len(feat)):

            layer_feat = feat[i]

            # hack to detect lower resolution
            if i>0 and feat[i].size(2) < feat[i-1].size(2):
                xx = xx/2.0
                yy = yy/2.0

            xx = np.clip(xx, 0, layer_feat.shape[2]-1).astype(np.int32)
            yy = np.clip(yy, 0, layer_feat.shape[3]-1).astype(np.int32)

            features = layer_feat[:,:, xx[range(samples)], yy[range(samples)]]
            feat_samples.append(features.clone().detach())

        feat = torch.cat(feat_samples,1)
        return feat
    
    def forward_cat(self, X, r, samps=100):
        feat = self.forward(X)

        try:
            r = r[:,:,0]
        except:
            pass

        if r.max()<0.1:
            region_mask = np.greater(r.flatten()+1.,0.5)
        else:
            region_mask = np.greater(r.flatten(),0.5)

        xx,xy = np.meshgrid(np.arange(X.shape[2]), np.arange(X.shape[3]))
        xx = np.expand_dims(xx.flatten(),1)
        xy = np.expand_dims(xy.flatten(),1)
        xc = np.concatenate([xx,xy],1)

        xc = xc[region_mask,:]

        samples = min(samps,xc.shape[0])

        np.random.shuffle(xc)
        xx = xc[:samples,0]
        yy = xc[:samples,1]

        feat_samples = []
        for i in range(len(feat)):
            layer_feat = feat[i]
            # hack to detect lower resolution
            if i>0 and feat[i].size(2) < feat[i-1].size(2):
                xx = xx/2.0
                yy = yy/2.0
            
            xx = np.clip(xx, 0, layer_feat.shape[2]-1).astype(np.int32)
            yy = np.clip(yy, 0, layer_feat.shape[3]-1).astype(np.int32)

            features = layer_feat[:,:, xx[range(samples)], yy[range(samples)]]
            feat_samples.append(features.clone().detach())

        feat = torch.cat(feat_samples,1)
        return feat

In [16]:
def optimize(result, content, style, content_path, style_path, scale, content_weight, lr, extractor, regions=0):
    # torch.autograd.set_detect_anomaly(True)
    result_pyramid = make_laplace_pyramid(result, 5)
    result_pyramid = [l.data.requires_grad_() for l in result_pyramid]

    opt_iter = 200

    # use rmsprop
    optimizer = optim.RMSprop(result_pyramid, lr=lr)

    # extract features for content
    feat_content = extractor(content) # 

    stylized = fold_laplace_pyramid(result_pyramid)
    
    # some inner loop that extracts samples
    feat_style_all = []
    for ri in range(len(regions[0])):
        r_temp = regions[0][ri]
        if len(r_temp.shape) > 2:
            r_temp = r_temp[:,:,0]
        
        r_temp = torch.from_numpy(r_temp).unsqueeze(0).unsqueeze(0).contiguous()
        r = tensor_resample(r_temp,[style.size(3),style.size(2)])[0,0,:,:].numpy()
        feat_style = None
        for j in range(5):
            with torch.no_grad():
                feat_e = extractor.forward_cat(style, r, samps=1000)  
                feat_style = feat_e if feat_style is None else torch.cat((feat_style, feat_e), dim=2)
        feat_style_all.append(feat_style)

    xx = {}
    xy = {}
    yx = {}
        
    for ri in range(len(regions[0])):

        try:
            temp = xx[ri]
        except:
            xx[ri] = []
            xy[ri] = []
            yx[ri] = []

        r_temp = regions[0][ri]
        r_temp = torch.from_numpy(r_temp).unsqueeze(0).unsqueeze(0).contiguous()
        r = tensor_resample(r_temp, ([stylized.size(3), stylized.size(2)]))[0,0,:,:].numpy()     

        if r.max()<0.1:
            r = np.greater(r+1.,0.5)
        else:
            r = np.greater(r,0.5)
   
        sample_indices(feat_content, feat_style_all, r, ri, xx, xy, yx) # 0 to sample over first layer extracted

    # init indices to optimize over
    # xx, xy = sample_indices(feat_content[0], feat_style) # 0 to sample over first layer extracted
    for it in range(opt_iter):
        optimizer.zero_grad()
        stylized = fold_laplace_pyramid(result_pyramid)
        # original code has resample here, seems pointless with uniform shuffle
        # ...
        # also shuffle them every y iter
        if it % 1 == 0 and it != 0:
            for ri in xx.keys():
                np.random.shuffle(xx[ri][0])
                np.random.shuffle(xy[ri][0])
                np.random.shuffle(yx[ri][0])

        feat_result = extractor(stylized)

        loss = calculate_loss(feat_result, feat_content, feat_style_all, xx, xy, yx, content_weight, regions)
        
        loss.backward()
        optimizer.step()
    
    return stylized

In [17]:
def strotss(content_pil, style_pil, content_path, style_path, regions, content_weight=1.0*16.0, device='cuda:0', space='uniform', content_mask=None, style_mask=None):
    content_np = pil_to_np(content_pil)
    style_np = pil_to_np(style_pil)
    content_full = np_to_tensor(content_np, space).to(device)
    style_full = np_to_tensor(style_np, space).to(device)

    if content_mask is not None and style_mask is not None:
        content_mask = content_mask.to(device)
        style_mask = style_mask.to(device)

    lr = 2e-3
    extractor = Vgg16_Extractor(space=space).to(device)

    scale_last = max(content_full.shape[2], content_full.shape[3])
    scales = []
    for scale in range(10):
        divisor = 2**scale
        if min(content_pil.width, content_pil.height) // divisor >= 33:
            scales.insert(0, divisor)
    
    for scale in scales:
        # rescale content to current scale
        content = tensor_resample(content_full, [ content_full.shape[2] // scale, content_full.shape[3] // scale ])
        style = tensor_resample(style_full, [ style_full.shape[2] // scale, style_full.shape[3] // scale ])
        print(f'Optimizing at resoluton [{content.shape[2]}, {content.shape[3]}]')

        # upsample or initialize the result
        if scale == scales[0]:
            # first
            result = laplacian(content) + style.mean(2,keepdim=True).mean(3,keepdim=True)
        elif scale == scales[-1]:
            # last 
            result = tensor_resample(result, [content.shape[2], content.shape[3]])
            lr = 1e-3
        else:
            result = tensor_resample(result, [content.shape[2], content.shape[3]]) + laplacian(content)

        # do the optimization on this scale
        result = optimize(result, content, style, content_path, style_path, scale, content_weight=content_weight, lr=lr, extractor=extractor, regions=regions)

        # next scale lower weight
        content_weight /= 2.0

    clow = -1.0 if space == 'uniform' else -1.7
    chigh = 1.0 if space == 'uniform' else 1.7
    result_image = tensor_to_np(tensor_resample(torch.clamp(result, clow, chigh), [content_full.shape[2], content_full.shape[3]])) # 
    # renormalize image
    result_image -= result_image.min()
    result_image /= result_image.max()
    return np_to_pil(result_image * 255.)

In [21]:
args={"content": "content.jpg", "style": "style.jpg", "content_mask": None, "style_mask": None, "weight": 1.0, "output": "strotss.png", "device": "cuda:0", "ospace": "uniform", "resize_to": 512}
args["content"] ="/Users/xiongjiangkai/Downloads/cv_heol/test_content/Tuebingen_Neckarfront.jpg"
args["style"] = "/Users/xiongjiangkai/Downloads/cv_heol/test_style/vangogh_starry_night.jpg"
args["output"]=args["content"].split('/')[-1].split('.')[0]+"_"+args["style"].split('/')[-1].split('.')[0]+".png"

# make 256 the smallest possible long side, will still fail if short side is <
if args["resize_to"] < 2**8:
    print("Resulution too low.")
    exit(1)

content_pil, style_pil = pil_loader(args["content"]), pil_loader(args["style"])
content_mask, style_mask = None, None

if args["content_mask"] and args["style_mask"] is not None:
    regions = extract_regions(args["content_mask"], args["style_mask"])

    pil_content_mask = pil_loader(args["content_mask"])
    pil_style_mask = pil_loader(args["style_mask"])

    pil_content_mask = pil_resize_long_edge_to(pil_content_mask, args["resize_to"])
    pil_style_mask = pil_resize_long_edge_to(pil_style_mask, args["resize_to"])

    content_mask = pil_to_np(pil_content_mask)
    style_mask = pil_to_np(pil_style_mask)

    content_mask = create_mask_from_image(content_mask)
    style_mask = create_mask_from_image(style_mask)
else:
    try:
        regions = [[pil_to_np(pil_resize_long_edge_to(pil_loader(args["content"]), args["resize_to"]))[:,:,0]*0.+1.], [pil_to_np(pil_resize_long_edge_to(pil_loader(args["style"]), args["resize_to"]))[:,:,0]*0.+1.]]
    except:
        regions = [[pil_to_np(pil_resize_long_edge_to(pil_loader(args["content"]), args["resize_to"]))[:,:]*0.+1.], [pil_to_np(pil_resize_long_edge_to(pil_loader(args["style"]), args["resize_to"]))[:,:]*0.+1.]]

content_weight = args["weight"] * 16.0

device = "mps"

start = time()
result = strotss(pil_resize_long_edge_to(content_pil, args["resize_to"]), 
                    pil_resize_long_edge_to(style_pil, args["resize_to"]), args["content"], args["style"], regions, content_weight, device, args["ospace"], content_mask=content_mask, style_mask=style_mask)
result.save(args["output"])
print(f'Done in {time()-start:.3f}s')

/Users/xiongjiangkai/.virtualenvs/.Data_science/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/xiongjiangkai/.virtualenvs/.Data_science/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Optimizing at resoluton [48, 64]
Style: 1.668, Content: 0.089
Style: 1.468, Content: 0.062
Style: 1.442, Content: 0.042
Style: 1.395, Content: 0.037
Style: 1.382, Content: 0.033
Style: 1.342, Content: 0.035
Style: 1.368, Content: 0.029
Style: 1.322, Content: 0.031
Style: 1.328, Content: 0.028
Style: 1.315, Content: 0.029
Style: 1.310, Content: 0.027
Style: 1.302, Content: 0.028
Style: 1.280, Content: 0.027
Style: 1.316, Content: 0.026
Style: 1.250, Content: 0.029
Style: 1.322, Content: 0.025
Style: 1.251, Content: 0.028
Style: 1.285, Content: 0.024
Style: 1.235, Content: 0.025
Style: 1.264, Content: 0.023
Style: 1.243, Content: 0.024
Style: 1.243, Content: 0.023
Style: 1.225, Content: 0.024
Style: 1.232, Content: 0.024
Style: 1.223, Content: 0.024
Style: 1.221, Content: 0.023
Style: 1.230, Content: 0.023
Style: 1.218, Content: 0.023
Style: 1.228, Content: 0.022
Style: 1.195, Content: 0.024
Style: 1.212, Content: 0.022
Style: 1.216, Content: 0.023
Style: 1.207, Content: 0.022
Style: 1.2